In [ ]:
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv

dotenv.load_dotenv(".env.local")
# dotenv.load_dotenv(".env.kpzdev")

from test_unit.tasks_unit1 import UnitTask1
from mrp_functions.base_tasks import TaskBase
from mrp_functions import logging
import re

np.__version__

part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(int(os.environ['SIMULATION']))
vehicle_color_code=os.environ['VEHICLE_COLOR_CODE']

tb = TaskBase()
tb.db_connection_health_check()

In [ ]:
# Validate MRP Data Before Running
tb.query_msc_volume_from_production_plan(production_plan_id, msc_code, vehicle_color_code,
                                         mrp_run_date, plant_code, count=True)

In [ ]:
# Get end date of production plan
end_date_of_prod_plan = tb.query_end_date_from_production_plan(production_plan_id)
if end_date_of_prod_plan is None:
    raise ValueError("Production Plan End Date is None!")
# Get list of all date from N to N+6 to end of production Plan
list_mrp_date = tb.query_list_date_from_Nplus1_to_Nplus6_definition(mrp_run_date, end_date_of_prod_plan)
print(f"[{msc_code}_{production_plan_id}_{mrp_run_date}_{plant_code}] Inventory Forecast From: {list_mrp_date[0]} To: {list_mrp_date[-1]}")


In [ ]:
list_msc_plant_code = tb.query_list_distinct_msc_from_production_plan(production_plan_id)
print(f'[{msc_code}_{production_plan_id}_{mrp_run_date}_{plant_code}] Number of MSCs from Production Plan: {len(list_msc_plant_code)}')
pd.DataFrame(list_msc_plant_code, columns=['MSC', 'Vehicle Color', 'Plant Code'])

In [ ]:
# For each MSC, to query all part required from BOM Master Table
each_msc_parts_need_frame = tb.query_parts_of_msc_from_boms(msc_code, plant_code)
# each_msc_parts_need_frame.loc[each_msc_parts_need_frame['part_code'] == '994630620B']
each_msc_parts_need_frame

In [ ]:
# For each MSC, to query all production volume from Production Table
production_volume = tb.query_msc_volume_from_production_plan(production_plan_id, msc_code, vehicle_color_code,
                                                             mrp_run_date, plant_code)
production_volume

In [ ]:
# Get list of unique vehicle color
msc_list_distinct_vehicle_color = production_volume.vehicle_color_code.unique().tolist()
msc_list_distinct_vehicle_color

In [ ]:
# 3 Filter Part Color Code XX
parts_with_color_xx = each_msc_parts_need_frame.loc[each_msc_parts_need_frame['part_color_code'] == 'XX']
parts_with_color_xx

In [ ]:
# Find and replace Color XX with Color Code found in Database
each_msc_parts_need_frame = tb.query_parts_of_msc_from_boms(msc_code, plant_code)
# each_msc_parts_need_frame['vehicle_color_code'] = np.NaN
# for part_code in parts_with_color_xx['part_code'].unique().tolist():
for part_color_xx_idx in parts_with_color_xx.index:
    for vehicle_color_code in msc_list_distinct_vehicle_color:
        part_code = parts_with_color_xx.loc[part_color_xx_idx, ['part_code']].item()
        color_code_for_xx = tb.query_part_color_xx(part_code, vehicle_color_code, plant_code)
        if color_code_for_xx is not None:
            # Assign color code for XX
            each_msc_parts_need_frame.loc[part_color_xx_idx, ['part_color_code']] = color_code_for_xx

each_msc_parts_need_frame

In [ ]:
print(type(each_msc_parts_need_frame['bom_ecn_in_date'][0]))
print(type(production_volume.iloc[0].plan_date))

In [ ]:
volume_parts_list_2 = []

for idx in production_volume.index:
    row = production_volume.iloc[idx]
    vf = each_msc_parts_need_frame.loc[((pd.isna(each_msc_parts_need_frame['bom_ecn_in_date'])) | (each_msc_parts_need_frame['bom_ecn_in_date'] <= row.plan_date)) &
                                      ((pd.isna(each_msc_parts_need_frame['bom_ecn_out_date'])) | (each_msc_parts_need_frame['bom_ecn_out_date'] >= row.plan_date)) &
                                      ((pd.isna(each_msc_parts_need_frame['part_ecn_in_date'])) | (each_msc_parts_need_frame['part_ecn_in_date'] <= row.plan_date)) &
                                      ((pd.isna(each_msc_parts_need_frame['part_ecn_out_date'])) | (each_msc_parts_need_frame['part_ecn_out_date'] >= row.plan_date)),
                                       ["msc_code", "part_code", "part_color_code", "part_quantity", "plant_code"]].copy(deep=True)
    if isinstance(vf, pd.Series):
        vf['production_date'] = row.plan_date
        vf['vehicle_color_code'] = row.vehicle_color_code
        vf['production_volume'] = row.volume
        vf['part_requirement_quantity'] = vf['part_quantity'] * row.volume
        list_f = [vf.to_dict()]
    else:
        vf['part_requirement_quantity'] = vf[['part_quantity']] * row.volume
        vf.loc[:, ['production_date']] = row.plan_date
        vf.loc[:, ['vehicle_color_code']] = row.vehicle_color_code
        vf.loc[:, ['production_volume']] = row.volume
        list_f = vf.to_dict(orient='records')
    volume_parts_list_2.extend(list_f)
# ht = pd.DataFrame(volume_parts_list_2)
# ht.loc[(ht['part_code'] == '994630620B') & (ht['production_date'] == '2022-09-18')]
pd.DataFrame(volume_parts_list_2)

In [ ]:
# to save MRP Results
mrp_parts_quantity = pd.DataFrame(volume_parts_list_2)
mrp_parts_quantity = mrp_parts_quantity.loc[~mrp_parts_quantity.duplicated(keep='first')]
mrp_parts_quantity[['import_id']] = production_plan_id
mrp_parts_quantity[['created_by']] = user_code
mrp_parts_quantity[['updated_by']] = user_code
# parent_start_timestamp = time.time()
# # tb.store_mrp_results(mrp_parts_quantity.to_dict(orient="records"), simulation)
#
# logging.info(f"[{msc_code}_{production_plan_id}_{mrp_run_date}_{plant_code}] "
#              f"Store MRP Result finished after: {time.time() - parent_start_timestamp:.2f} sec")

mrp_parts_quantity


In [ ]:
# Store MRP to DB
# Convert List of Series to DataFrame
mrp_parts_quantity = pd.DataFrame(volume_parts_list).set_index(
    ['production_date', 'msc_code', 'vehicle_color_code', 'production_volume']).fillna(0.0)
# Drop Multi-Index vehicle_color_code
mrp_parts_quantity = mrp_parts_quantity.droplevel(10, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(9, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(8, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(7, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(6, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(5, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(4, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(3, axis=1)
mrp_parts_quantity = mrp_parts_quantity.droplevel(2, axis=1)

# mrp_parts_quantity = mrp_parts_quantity.loc[
#     mrp_parts_quantity.index.get_level_values('production_date') > pd.to_datetime((datetime.strptime(mrp_run_date, "%Y-%m-%d")))]

mrp_parts_quantity

In [ ]:
mrp_parts_quantity = mrp_parts_quantity.loc[:, ~mrp_parts_quantity.columns.duplicated(keep='first')]
mrp_parts_quantity

In [ ]:
mrp_parts_quantity = mrp_parts_quantity.stack().reset_index().set_index([
    'production_date', 'production_volume', 'msc_code', 'vehicle_color_code', 'part_color_code']).stack().reset_index()
mrp_parts_quantity.rename(columns={mrp_parts_quantity.columns[-1]: 'part_requirement_quantity'}, inplace=True)
mrp_parts_quantity

In [ ]:
mrp_parts_quantity.loc[mrp_parts_quantity['part_code'] == '9946JU16B']

In [ ]:
# to save MRP Results
mrp_parts_quantity[['import_id']] = production_plan_id
mrp_parts_quantity[['plant_code']] = plant_code
mrp_parts_quantity[['created_by']] = user_code
mrp_parts_quantity[['updated_by']] = user_code

parent_start_timestamp = time.time()
# tb.store_mrp_results(mrp_parts_quantity.to_dict(orient="records"), simulation)

logging.info(f"[{msc_code}_{production_plan_id}_{mrp_run_date}_{plant_code}] "
             f"Store MRP Result finished after: {time.time() - parent_start_timestamp:.2f} sec")
